In [1]:
from manim import *

Manim Community v0.16.0.post0

In [2]:
from deepgd import *
from deepgd.tools import *
from deepgd.utils import *
from deepgd.modules import *
from deepgd.data import *
from deepgd.xing import *
from deepgd.transform import *

In [3]:
G_list = load_G_list(data_path='data/rome', index_file='data_index.txt', cache='G_list', cache_prefix='cache/')

Load from 'cache/G_list.pickle'


In [4]:
data_list = generate_data_list(G_list, 
                               sparse=False, 
                               pivot_mode=None,
                               init_mode='pmds',
                               edge_index='full_edge_index',
                               edge_attr='full_edge_attr',
                               pmds_list=np.load('layouts/rome/pmds.npy', allow_pickle=True),
                               gviz_list=np.load('layouts/rome/gviz.npy', allow_pickle=True),
                               noisy_layout=True,
                               device='cpu', 
                               cache=True,
                               cache_prefix='cache/')

Load from 'cache/generate_data_list(list,sparse=False,pivot_mode=None,init_mode=pmds,edge_index=full_edge_index,edge_attr=full_edge_attr,pmds_list=ndarray,gviz_list=ndarray,noisy_layout=True,device=cpu).pickle'


In [5]:
idx = 422
graph_scale = 12
init_scale = 2
frame_scale = 1.5
start_x, end_x = -4, 4
colors = [WHITE, YELLOW, RED, GREEN]
aggr_lag = 0.15
ending_wait = 2

In [6]:
normalize = Canonicalization(rotate=RotateByPrincipalComponents(base_angle=-np.pi/2),
                             normalize=Normalization(scale_factor=graph_scale/np.sqrt(100)))
shear = LinearTransformation(torch.tensor([[1/2, 0], 
                                           [1/2, 1]]))

In [7]:
%%manim -r 500,500 Graph
class Graph(Scene):
    def construct(self):
        n_layers = len(colors)
        interval = (end_x-start_x)/(n_layers-1)
        edges = torch.tensor(list(G_list[idx].edges))
        raw_pos = normalize(data_list[idx].gt_pos, data_list[idx])
        pos = shear(raw_pos, data_list[idx])
        layer = Polygon(*((np.array([[-1, -3, 0],
                                     [1, -1, 0],
                                     [1, 3, 0],
                                     [-1, 1, 0]])*frame_scale).tolist()), 
                        stroke_color=GREY, 
                        fill_color=GREY).set_opacity(0.5).shift(RIGHT*start_x)
        begin_lines = [Line([*p, 0], [*q, 0]).set_color(GRAY) for p, q in raw_pos[edges]*init_scale]    
        begin_dots = [Dot().shift([*p, 0]).set_color(colors[0]).scale(init_scale) for p in raw_pos*init_scale]
        lines = [Line([*p, 0], [*q, 0]).shift(RIGHT*start_x).set_color(GRAY) for p, q in pos[edges]]
        dots = [Dot().shift([*p, 0]).shift(RIGHT*start_x).set_color(colors[0]) for p in pos]
        end_lines = [Line([*p, 0], [*q, 0]).set_color(GRAY) for p, q in raw_pos[edges]*init_scale]    
        end_dots = [Dot().shift([*p, 0]).set_color(colors[-1]).scale(init_scale) for p in raw_pos*init_scale]    
        
        self.play(FadeIn(layer),
                  *[Transform(begin_line, line) for begin_line, line in zip(begin_lines, lines)],
                  *[Transform(begin_dot, dot) for begin_dot, dot in zip(begin_dots, dots)])
        
        groups = [[layer] + begin_lines + lines + begin_dots + dots]
        for x_pos, color in zip(np.linspace(start_x, end_x, n_layers)[:-1], colors[1:]):
            all_msg_dots = []
            animations = []
            for c_idx in range(len(pos)):
                msg_pos = pos[torch.tensor(list(dict(G_list[idx].adjacency())[c_idx].keys()))]
                msg_dots = [Dot().shift([*p, 0]).shift(RIGHT*x_pos).set_fill(color, opacity=0) for p in msg_pos]
                msg_shifts = [[pos[c_idx][0]-x, pos[c_idx][1]-y, 0] for x, y in msg_pos]
                all_msg_dots += msg_dots

    #             self.play(*[dot.animate.shift(shift).set_opacity(1) for dot, shift in zip(msg_dots, msg_shifts)])
                animations.append(AnimationGroup(
                    *[dot.animate.shift(shift).set_opacity(1) for dot, shift in zip(msg_dots, msg_shifts)]
                ))
            self.play(AnimationGroup(*animations, lag_ratio=aggr_lag))
            [d.set_opacity(0) for d in all_msg_dots]

            layer = Polygon(*((np.array([[-1, -3, 0],
                                         [1, -1, 0],
                                         [1, 3, 0],
                                         [-1, 1, 0]])*frame_scale).tolist()), 
                            stroke_color=GREY, 
                            fill_color=GREY).set_opacity(0.5).shift(RIGHT*(x_pos+interval))
            lines = [l.copy() for l in lines]
            dots = [d.copy().set_color(color) for d in dots]
            groups.append([layer] + lines + dots + all_msg_dots)
            
            self.play(FadeIn(layer),
                      *[l.animate.shift(RIGHT*interval) for l in lines],
                      *[d.animate.shift(RIGHT*interval) for d in dots]) 
            
        
        lines = [l.copy() for l in lines]
        dots = [d.copy().set_color(colors[-1]) for d in dots]
        self.add(*lines)
        self.add(*dots)
        
        self.play(*[FadeOut(obj) for group in groups for obj in group])
        self.play(*[Transform(line, end_line) for line, end_line in zip(lines, end_lines)],
                  *[Transform(dot, end_dot) for dot, end_dot in zip(dots, end_dots)])
        self.play(Wait().set_run_time(ending_wait))

[08/20/22 22:26:07] INFO     Animation 0 : Partial movie file        ]8;id=427464;file:///opt/homebrew/anaconda3/lib/python3.9/site-packages/manim/scene/scene_file_writer.py\scene_file_writer.py]8;;\:]8;id=883098;file:///opt/homebrew/anaconda3/lib/python3.9/site-packages/manim/scene/scene_file_writer.py#514\514]8;;\
                             written in '/Users/yolanda/Desktop/Rese                         
                             arch/DeepGD/VIS Present/deepgd/media/vi                         
                             deos/deepgd/500p60/partial_movie_files/                         
                             Graph/1090121337_1521949980_758648948.m                         
                             p4'                                                             

                    INFO     Animation 1 : Partial movie file        ]8;id=899498;file:///opt/homebrew/anaconda3/lib/python3.9/site-packages/manim/scene/scene_file_writer.py\scene_file_writer.py]8;;\:]8;id=332445;file:///opt/homebrew/anaconda3/lib/python3.9/site-packages/manim/scene/scene_file_writer.py#514\514]8;;\
                             written in '/Users/yolanda/Desktop/Rese                         
                             arch/DeepGD/VIS Present/deepgd/media/vi                         
                             deos/deepgd/500p60/partial_movie_files/                         
                             Graph/1222978634_738743836_918495816.mp                         
                             4'                                                              

[08/20/22 22:26:08] INFO     Animation 2 : Partial movie file        ]8;id=819617;file:///opt/homebrew/anaconda3/lib/python3.9/site-packages/manim/scene/scene_file_writer.py\scene_file_writer.py]8;;\:]8;id=423192;file:///opt/homebrew/anaconda3/lib/python3.9/site-packages/manim/scene/scene_file_writer.py#514\514]8;;\
                             written in '/Users/yolanda/Desktop/Rese                         
                             arch/DeepGD/VIS Present/deepgd/media/vi                         
                             deos/deepgd/500p60/partial_movie_files/                         
                             Graph/1222978634_1881574975_2487118387.                         
                             mp4'                                                            

[08/20/22 22:26:09] INFO     Animation 3 : Partial movie file        ]8;id=71668;file:///opt/homebrew/anaconda3/lib/python3.9/site-packages/manim/scene/scene_file_writer.py\scene_file_writer.py]8;;\:]8;id=484613;file:///opt/homebrew/anaconda3/lib/python3.9/site-packages/manim/scene/scene_file_writer.py#514\514]8;;\
                             written in '/Users/yolanda/Desktop/Rese                         
                             arch/DeepGD/VIS Present/deepgd/media/vi                         
                             deos/deepgd/500p60/partial_movie_files/                         
                             Graph/1222978634_898754443_4195782805.m                         
                             p4'                                                             

[08/20/22 22:26:10] INFO     Animation 4 : Partial movie file        ]8;id=613189;file:///opt/homebrew/anaconda3/lib/python3.9/site-packages/manim/scene/scene_file_writer.py\scene_file_writer.py]8;;\:]8;id=724342;file:///opt/homebrew/anaconda3/lib/python3.9/site-packages/manim/scene/scene_file_writer.py#514\514]8;;\
                             written in '/Users/yolanda/Desktop/Rese                         
                             arch/DeepGD/VIS Present/deepgd/media/vi                         
                             deos/deepgd/500p60/partial_movie_files/                         
                             Graph/1222978634_3925929047_1751491064.                         
                             mp4'                                                            

...

                    INFO     Animation 9 : Partial movie file        ]8;id=42892;file:///opt/homebrew/anaconda3/lib/python3.9/site-packages/manim/scene/scene_file_writer.py\scene_file_writer.py]8;;\:]8;id=907198;file:///opt/homebrew/anaconda3/lib/python3.9/site-packages/manim/scene/scene_file_writer.py#514\514]8;;\
                             written in '/Users/yolanda/Desktop/Rese                         
                             arch/DeepGD/VIS Present/deepgd/media/vi                         
                             deos/deepgd/500p60/partial_movie_files/                         
                             Graph/1222978634_2464843671_1581323329.                         
                             mp4'                                                            

                    INFO     Combining to Movie file.                ]8;id=458225;file:///opt/homebrew/anaconda3/lib/python3.9/site-packages/manim/scene/scene_file_writer.py\scene_file_writer.py]8;;\:]8;id=452847;file:///opt/homebrew/anaconda3/lib/python3.9/site-packages/manim/scene/scene_file_writer.py#607\607]8;;\

                    INFO                                             ]8;id=802291;file:///opt/homebrew/anaconda3/lib/python3.9/site-packages/manim/scene/scene_file_writer.py\scene_file_writer.py]8;;\:]8;id=520108;file:///opt/homebrew/anaconda3/lib/python3.9/site-packages/manim/scene/scene_file_writer.py#728\728]8;;\
                             File ready at '/Users/yolanda/Desktop/R                         
                             esearch/DeepGD/VIS Present/deepgd/media                         
                             /videos/deepgd/500p60/Graph.mp4'                                
                                                                                             

                    INFO     The partial movie directory is full (>  ]8;id=190174;file:///opt/homebrew/anaconda3/lib/python3.9/site-packages/manim/scene/scene_file_writer.py\scene_file_writer.py]8;;\:]8;id=975847;file:///opt/homebrew/anaconda3/lib/python3.9/site-packages/manim/scene/scene_file_writer.py#699\699]8;;\
                             100 files). Therefore, manim has                                
                             removed the 10 oldest file(s). You can                          
                             change this behaviour by changing                               
                             max_files_cached in config.                                     

                    INFO     Rendered Graph                                      ]8;id=824379;file:///opt/homebrew/anaconda3/lib/python3.9/site-packages/manim/scene/scene.py\scene.py]8;;\:]8;id=212050;file:///opt/homebrew/anaconda3/lib/python3.9/site-packages/manim/scene/scene.py#240\240]8;;\
                             Played 10 animations                                            

In [8]:
%%manim TwoCircles
class TwoCircles(Scene):
    def construct(self):
        r = Circle(color=RED, fill_opacity=1, z_index=0).shift(RIGHT)
        g = Circle(color=GREEN, fill_opacity=1, z_index=1)
        self.add(r)
        self.add(g)
        self.play(r.animate.shift(LEFT))
        self.play(LaggedStart(r.animate.shift(LEFT)))

[08/20/22 22:26:15] INFO     Animation 0 : Partial movie file        ]8;id=180514;file:///opt/homebrew/anaconda3/lib/python3.9/site-packages/manim/scene/scene_file_writer.py\scene_file_writer.py]8;;\:]8;id=638837;file:///opt/homebrew/anaconda3/lib/python3.9/site-packages/manim/scene/scene_file_writer.py#514\514]8;;\
                             written in '/Users/yolanda/Desktop/Rese                         
                             arch/DeepGD/VIS Present/deepgd/media/vi                         
                             deos/deepgd/1080p60/partial_movie_files                         
                             /TwoCircles/2724823860_3556240383_18910                         
                             98068.mp4'                                                      

[08/20/22 22:26:16] INFO     Animation 1 : Partial movie file        ]8;id=302549;file:///opt/homebrew/anaconda3/lib/python3.9/site-packages/manim/scene/scene_file_writer.py\scene_file_writer.py]8;;\:]8;id=87322;file:///opt/homebrew/anaconda3/lib/python3.9/site-packages/manim/scene/scene_file_writer.py#514\514]8;;\
                             written in '/Users/yolanda/Desktop/Rese                         
                             arch/DeepGD/VIS Present/deepgd/media/vi                         
                             deos/deepgd/1080p60/partial_movie_files                         
                             /TwoCircles/1164547788_20467504_2967172                         
                             018.mp4'                                                        

                    INFO     Combining to Movie file.                ]8;id=289707;file:///opt/homebrew/anaconda3/lib/python3.9/site-packages/manim/scene/scene_file_writer.py\scene_file_writer.py]8;;\:]8;id=691994;file:///opt/homebrew/anaconda3/lib/python3.9/site-packages/manim/scene/scene_file_writer.py#607\607]8;;\

                    INFO                                             ]8;id=83498;file:///opt/homebrew/anaconda3/lib/python3.9/site-packages/manim/scene/scene_file_writer.py\scene_file_writer.py]8;;\:]8;id=728852;file:///opt/homebrew/anaconda3/lib/python3.9/site-packages/manim/scene/scene_file_writer.py#728\728]8;;\
                             File ready at '/Users/yolanda/Desktop/R                         
                             esearch/DeepGD/VIS Present/deepgd/media                         
                             /videos/deepgd/1080p60/TwoCircles.mp4'                          
                                                                                             

                    INFO     Rendered TwoCircles                                 ]8;id=960976;file:///opt/homebrew/anaconda3/lib/python3.9/site-packages/manim/scene/scene.py\scene.py]8;;\:]8;id=168503;file:///opt/homebrew/anaconda3/lib/python3.9/site-packages/manim/scene/scene.py#240\240]8;;\
                             Played 2 animations                                             